<a href="https://colab.research.google.com/github/wonjae124/Capston/blob/main/RandomizedSearchCV_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning/multiple_classification/Iris.csv')

In [3]:
x_data_df = df.drop(['Id','Species'],axis = 1) #drop 메서드로 해당하는 값(DataFrame에서는 열)이 삭제 후 반환. https://stml.tistory.com/26 
y_data_df = df['Species'] #정답 라벨이다.

print(x_data_df.head(),'\n')

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2 



In [4]:
print(y_data_df)

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: Species, Length: 150, dtype: object


In [5]:
x_data = x_data_df.to_numpy()

In [6]:
le = LabelEncoder()
le.fit(y_data_df)
print(le.classes_)
labels = le.classes_ #파이썬 키워드와의 충돌을 피하기 위한 single_trailing_underscore이다. 문자 뒤에 붙는 underscore임.
y_data = le.transform(y_data_df) # transform 하면 0,1,2 이렇게 라벨 인코딩 성공

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 777, stratify=y_data) #stratify=target, Classification에서는 필수인 파라미터임. 각 분할 된 데이터셋의 class 비율을 orginal로 유지해준다. 
 
model = DecisionTreeClassifier()

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [7]:
print(len(y_data)) 

150


## 라벨 인코딩 결과.

In [8]:
y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 조정 가능한 model의 parameter 탐색

In [9]:
print(model.get_params().keys())

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])


In [10]:
param_distributions = {
    'criterion':['gini','entropy'],
    'max_depth':[None, 2, 3, 4, 5, 6]
}


randomized_search = RandomizedSearchCV(model, param_distributions = param_distributions, n_iter = 10, return_train_score=True, refit = True) #나머지 파라미터가 무슨 뜻?
#RandomizedSearchCV(estimator, param_distributions = 최적화를 찾을 매개변수 목록, n_iter=샘플링 된 매개변수 수, retun_train_score로 크로스 벨리데이션 모델 사용, refit=bool로 전체 데이터셋을 찾은 best 파라미터에 적용한다.)
randomized_search.fit(x_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 2, 3, 4, 5, 6]},
                   return_train_score=True)

In [11]:
df = pd.DataFrame(randomized_search.cv_results_).sort_values(by=['mean_test_score','mean_train_score'], ascending = False)

print(df[['params', 'mean_train_score', 'mean_test_score']])


                                        params  mean_train_score  \
1     {'max_depth': None, 'criterion': 'gini'}          1.000000   
2  {'max_depth': None, 'criterion': 'entropy'}          1.000000   
5     {'max_depth': 6, 'criterion': 'entropy'}          1.000000   
6        {'max_depth': 5, 'criterion': 'gini'}          1.000000   
8        {'max_depth': 4, 'criterion': 'gini'}          0.995238   
4        {'max_depth': 6, 'criterion': 'gini'}          1.000000   
7     {'max_depth': 5, 'criterion': 'entropy'}          1.000000   
0     {'max_depth': 3, 'criterion': 'entropy'}          0.978571   
9        {'max_depth': 3, 'criterion': 'gini'}          0.978571   
3     {'max_depth': 2, 'criterion': 'entropy'}          0.971429   

   mean_test_score  
1         0.961905  
2         0.961905  
5         0.961905  
6         0.961905  
8         0.961905  
4         0.952381  
7         0.952381  
0         0.942857  
9         0.942857  
3         0.923810  


In [16]:
print(randomized_search.best_params_)

print(randomized_search.best_score_)

{'max_depth': None, 'criterion': 'gini'}
0.9619047619047618


In [12]:
x_test = [
          [5.3 , 3.7, 1.5, 0.2]
]

y_predict = randomized_search.best_estimator_.predict(x_test)
print(labels[y_predict[0]])

Iris-setosa


In [13]:
model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

https://wikidocs.net/42732